In [15]:
%cd /content/drive/MyDrive/Github/FL-project

/content/drive/MyDrive/Github/FL-project


In [16]:
from main import *
import argparse
from centralized import Centralized
from client import *

In [17]:
class Args():
  def __init__(self, dataset, model, bs, hnm, num_epochs, lr):
    self.dataset = dataset
    self.model = model 
    self.bs = bs
    self.hnm = hnm
    self.num_epochs = num_epochs
    self.lr = lr
    

args = Args('iddaCB', model = 'deeplabv3_mobilenetv2', bs = 4, hnm = False, num_epochs = 2, lr=0.01)

In [18]:
trainAloneDataset, testAloneDatasets = get_datasets(args)
ds = trainAloneDataset[0]

In [19]:
#istanziamo il modello 
model = model_init(args)


In [20]:
centralized = Centralized(args,ds,model)
state_dict = torch.load('modelliSalvati/checkpoint.pth')
#model.classifier.load_state_dict(state_dict)
centralized.train()

params freezed
epoca 0
epoch 1 / 2, step 10 / 150, loss = 2.452
epoch 1 / 2, step 20 / 150, loss = 1.799
epoch 1 / 2, step 30 / 150, loss = 1.324
epoch 1 / 2, step 40 / 150, loss = 1.250
epoch 1 / 2, step 50 / 150, loss = 0.878
epoch 1 / 2, step 60 / 150, loss = 0.911
epoch 1 / 2, step 70 / 150, loss = 0.813
epoch 1 / 2, step 80 / 150, loss = 0.745
epoch 1 / 2, step 90 / 150, loss = 0.610
epoch 1 / 2, step 100 / 150, loss = 0.820
epoch 1 / 2, step 110 / 150, loss = 0.608
epoch 1 / 2, step 120 / 150, loss = 0.558
epoch 1 / 2, step 130 / 150, loss = 0.617
epoch 1 / 2, step 140 / 150, loss = 0.564
epoch 1 / 2, step 150 / 150, loss = 0.568
epoca 1
epoch 2 / 2, step 10 / 150, loss = 0.627
epoch 2 / 2, step 20 / 150, loss = 0.572
epoch 2 / 2, step 30 / 150, loss = 0.433
epoch 2 / 2, step 40 / 150, loss = 0.518
epoch 2 / 2, step 50 / 150, loss = 0.527
epoch 2 / 2, step 60 / 150, loss = 0.523
epoch 2 / 2, step 70 / 150, loss = 0.571


KeyboardInterrupt: ignored

In [ ]:
from utils.stream_metrics import StreamSegMetrics
metric = StreamSegMetrics(16, 'centralizzato')
centralized.test(metric)

In [ ]:
metric.get_results()['Mean IoU']

0.27598906127004535

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(ds, batch_size = 2, shuffle = False, drop_last = True)

In [ ]:
centralized.model.eval()
with torch.no_grad():
  for i, (images, labels) in enumerate(train_loader):
    images = images.cuda() 
    labels = labels.cuda()
    outputs = centralized.model(images)['out']
    break

In [ ]:
outputs = centralized.model(images[0].view(1, 3, 512, 928))['out']

In [ ]:
import matplotlib.pyplot as plt
def unNormalize(tensorImage, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        for t, m, s in zip(tensorImage, mean, std):
            t.mul_(s).add_(m)
        return tensorImage
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))

In [ ]:
singoloOutputLogits = outputs[0]
prediction = singoloOutputLogits.argmax(0)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))
plt.imshow(prediction.cpu().numpy(), alpha = 0.4)

In [ ]:
import matplotlib as mpl

plt.imshow(labels[0].cpu().numpy(), cmap = mpl.colormaps['inferno'], vmin = 0, vmax = 16)